In [2]:
#!/usr/bin/python3
import datetime
# 存取當下的日期(年月日)
datetime.datetime.now()
now_year, now_month, now_day = str(datetime.datetime.now()).split(' ')[0].split('-')
timestamp_now = int(datetime.datetime(int(now_year), int(now_month), int(now_day), 0, 0).timestamp())

#now_year, now_month, now_day 

In [3]:
# 製造該ETF當下的網址(from yahoo finance)
def url_of_ETF_now(etf_name):
    url='https://finance.yahoo.com/quote/ETF_NAME/history?period1=1448899200&period2=NOW_TIME&interval=1d&filter=history&frequency=1d.html'
   
    exact_url=url.replace('ETF_NAME', str(etf_name))
    exact_url=exact_url.replace('NOW_TIME', str(timestamp_now))
    
    return exact_url
#url_of_ETF_now('LQD')

In [4]:
# 讀標的ETF的名子
import csv

goal_ETF = []
with open('goal_ETF.csv', newline='') as f:
    reader = csv.reader(f)
    for row in reader:
        #print(row[0])
        goal_ETF.append(row[0])
print(goal_ETF)

['LQD', 'VCSH', 'VCIT', 'IGSB', 'IGIB', 'SPSB', 'SPIB', 'USIG', 'VCLT', 'GSY', 'SLQD', 'ICSH', 'CORP', 'IGLB', 'SPLB', 'IGHG', 'LQDH', 'QLTA', 'PFIG', 'SKOR', 'IGBH', 'CBND', 'LKOR', 'LDRI', 'IGIH']


In [5]:
import requests                    ## 先把套件 載入

# 使用 requests.get 把網頁內容載下來
def get_response(etf_name):
    response = requests.get(url_of_ETF_now(etf_name))
    #response.text  # 展示載來的資料
    # 使用 split 這個函數 把文字做切割成不同行
    lines = response.text.split('{"date":')
    return lines

#get_response('LQD')
# 觀察切割後想要的資料的格式 及長度
#print(lines[1])
#print(lines[100])
#print(lines[300].split('}')[0])
#len(lines[300].split('}')[0].split(','))

In [6]:
def get_price(etf_name):    #取得該ETF各日價格
    
    newlines = []     # 定義一個我真的想要的那些行
    price = []        # 收集價格
    
    for line in get_response(etf_name):
        line = line.split('}')[0]             # 取'}'前的部分
        if len(line.split(',')) == 7:        # 用「,」切欄位 看是否為7欄

            newlines.append(line)          # 將 line 加到 newlines 中

    for i in range(len(newlines)-1):
        if newlines[i].split(',')[0].isdigit():     # 檢查非空白 
            
            adj_price = round(float(newlines[i].split(',')[6].split(':')[1][:-1]),2)
            price.append(adj_price)
            
    return price
get_price('LQD')

[117.17,
 116.82,
 116.8,
 116.86,
 116.44,
 116.67,
 116.86,
 116.54,
 116.41,
 116.26,
 115.88,
 115.77,
 115.73,
 115.47,
 115.67,
 115.83,
 116.16,
 115.98,
 116.02,
 115.5,
 115.83,
 115.93,
 115.86,
 115.72,
 115.59,
 115.89,
 115.73,
 115.99,
 115.7,
 115.72,
 115.94,
 115.47,
 115.63,
 115.91,
 115.1,
 114.58,
 114.33,
 114.35,
 114.25,
 113.95,
 113.66,
 113.47,
 113.27,
 113.05,
 112.83,
 112.76,
 113.06,
 112.85,
 113.15,
 112.72,
 112.47,
 112.46,
 112.53,
 112.48,
 112.13,
 111.82,
 111.3,
 111.3,
 111.39,
 111.64,
 111.75,
 112.12,
 112.21,
 111.81,
 111.69,
 111.81,
 111.51,
 111.4,
 111.2,
 110.86,
 110.85,
 110.51,
 110.49,
 110.08,
 110.15,
 110.47,
 110.17,
 110.31,
 110.26,
 110.21,
 109.98,
 110.24,
 110.35,
 110.13,
 110.34,
 110.64,
 110.79,
 110.8,
 110.72,
 111.04,
 110.76,
 110.57,
 110.37,
 110.82,
 110.48,
 110.8,
 111.04,
 111.17,
 110.98,
 111.12,
 110.96,
 110.81,
 110.94,
 111.08,
 111.08,
 111.63,
 111.35,
 111.55,
 111.53,
 111.07,
 111.38,
 111.06,
 1

In [7]:
# 檢查各個ETF價格筆數是否相同 
from statistics import mode
day_numbers = []
for ETF in goal_ETF:       
    day_numbers.append(len(get_price(ETF)))
    print(len(get_price(ETF)))
                       
for ETF in goal_ETF:    # 印出有少資料的ETF
    if len(get_price(ETF)) != mode(day_numbers):
        print(ETF)

830
830
830
830
830
830
830
830
830
830
830
830
830
205
830
830
830
830
830
830
205
830
830
830
830
IGLB
IGBH


In [8]:
# 去除資料不足 (在2015/12/1後有出現沒資料的 有兩個)
goal_ETF.remove('IGLB')
goal_ETF.remove('IGBH')
#goal_ETF

In [9]:
# 存取總價格資料
Price_Total = {}

for ETF in goal_ETF:
    Price_Total[ETF] = get_price(ETF)

# 存取總日期資料


def Total_date(etf_name):
    date_Total = []
    lines = []
    
    for line in get_response(etf_name):
        line = line.split('}')[0]
        if len(line.split(',')) == 7:   
            lines.append(line) 
        
    for i in range(len(lines)-1):
        if lines[i].split(',')[0].isdigit():
            epoch_date = int(lines[i].split(',')[0])
            date_Total.append(str(datetime.datetime.fromtimestamp(epoch_date)).split(' ')[0])
    return date_Total
        

Total_date(goal_ETF[0])

['2019-03-21',
 '2019-03-19',
 '2019-03-18',
 '2019-03-15',
 '2019-03-14',
 '2019-03-13',
 '2019-03-12',
 '2019-03-11',
 '2019-03-08',
 '2019-03-07',
 '2019-03-06',
 '2019-03-05',
 '2019-03-04',
 '2019-03-01',
 '2019-02-28',
 '2019-02-27',
 '2019-02-26',
 '2019-02-25',
 '2019-02-22',
 '2019-02-21',
 '2019-02-20',
 '2019-02-19',
 '2019-02-15',
 '2019-02-14',
 '2019-02-13',
 '2019-02-12',
 '2019-02-11',
 '2019-02-08',
 '2019-02-07',
 '2019-02-06',
 '2019-02-05',
 '2019-02-04',
 '2019-02-01',
 '2019-01-31',
 '2019-01-30',
 '2019-01-29',
 '2019-01-28',
 '2019-01-25',
 '2019-01-24',
 '2019-01-23',
 '2019-01-22',
 '2019-01-18',
 '2019-01-17',
 '2019-01-16',
 '2019-01-15',
 '2019-01-14',
 '2019-01-11',
 '2019-01-10',
 '2019-01-09',
 '2019-01-08',
 '2019-01-07',
 '2019-01-04',
 '2019-01-03',
 '2019-01-02',
 '2018-12-31',
 '2018-12-28',
 '2018-12-27',
 '2018-12-26',
 '2018-12-24',
 '2018-12-21',
 '2018-12-20',
 '2018-12-19',
 '2018-12-18',
 '2018-12-17',
 '2018-12-14',
 '2018-12-13',
 '2018-12-

In [10]:
len(Total_date(goal_ETF[0]))

830

In [11]:
import pandas
from collections import OrderedDict

Total = OrderedDict(Price_Total)  
Total.update({'Date': Total_date(goal_ETF[0])})    # 加入日期
Total.move_to_end('Date',last = False)    # 把日期放最前面

df = pandas.DataFrame(Total)
df.head(20)

,Date,LQD,VCSH,VCIT,IGSB,IGIB,SPSB,SPIB,USIG,VCLT,...,SPLB,IGHG,LQDH,QLTA,PFIG,SKOR,CBND,LKOR,LDRI,IGIH
0,2019-03-21,117.26,79.38,85.97,52.56,54.53,30.46,33.91,54.59,8.00,...,26.46,74.00,93.22,51.38,25.13,50.10,31.38,50.00,24.87,23.21
1,2019-03-19,116.82,79.26,85.68,52.51,54.37,30.43,33.84,54.40,88.74,...,26.28,74.04,93.52,51.37,25.13,50.09,31.40,50.30,24.88,23.18
2,2019-03-18,116.80,79.24,85.65,52.49,54.36,30.42,33.83,54.40,88.68,...,26.31,73.92,93.36,51.36,25.14,50.19,31.41,50.46,24.80,23.19
3,2019-03-15,116.86,79.27,85.71,52.00,54.35,30.43,33.82,54.41,88.74,...,26.30,73.84,93.32,51.37,25.18,50.19,31.36,50.43,24.83,23.18
4,2019-03-14,116.44,79.17,85.43,52.44,54.20,30.41,33.77,54.26,88.29,...,26.15,74.16,93.36,51.24,25.14,50.11,31.31,50.30,24.93,23.09
5,2019-03-13,116.67,79.20,85.00,52.41,54.23,30.41,33.78,54.32,88.58,...,26.27,73.84,93.34,51.31,25.15,50.11,31.33,50.48,24.93,23.16
6,2019-03-12,116.86,79.16,85.56,52.44,54.21,30.42,33.80,54.40,88.83,...,26.36,73.78,93.00,51.38,25.18,50.16,31.32,50.48,24.88,23.13
7,2019-03-11,116.54,79.11,85.37,52.38,54.12,30.40,33.76,54.20,88.39,...,26.22,73.73,93.43,51.23,25.11,49.94,31.24,50.19,24.88,23.14
8,2019-03-08,116.41,79.10,85.28,52.33,54.02,30.38,33.73,54.19,88.28,...,26.18,73.86,93.20,51.21,25.13,49.94,31.24,50.03,24.92,23.14
9,2019-03-07,116.26,79.10,85.21,52.30,54.03,30.39,33.72,54.16,88.17,...,26.16,73.64,93.30,51.20,25.15,5.00,31.23,50.09,24.86,23.14


In [12]:
P_T_of_IGLB_IGBH = {}
for ETF in ['IGLB','IGBH']:
    P_T_of_IGLB_IGBH[ETF] = get_price(ETF)
Total_IGLB_IGBH = OrderedDict(P_T_of_IGLB_IGBH)  
Total_IGLB_IGBH.update({'Date': Total_date('IGLB')})    # 加入日期
Total_IGLB_IGBH.move_to_end('Date',last = False)    # 把日期放最前面

df = pandas.DataFrame(Total_IGLB_IGBH)
df.head(20)

,Date,IGLB,IGBH
0,2019-03-21,58.57,24.61
1,2019-03-19,58.42,24.67
2,2019-03-18,58.46,24.67
3,2019-03-15,58.48,24.65
4,2019-03-14,58.23,24.65
5,2019-03-13,58.42,24.66
6,2019-03-12,58.62,24.76
7,2019-03-11,58.32,24.71
8,2019-03-08,58.18,24.59
9,2019-03-07,58.12,24.66


In [13]:
import time
time.ctime(time.time())
str(datetime.datetime.now())

'2019-03-21 02:07:10.832582'